In [27]:
import numpy as np
import pandas as pd
import io
import os
from sklearn import metrics
from sklearn.feature_extraction.text import  CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline
import regex
import eventlet
from eventlet.green.urllib import request

In [28]:
book_files={
 "Mickiewicz": [
 "https://wolnelektury.pl/media/book/txt/dziady-dziady-widowisko-czesc-i.txt",
 "https://wolnelektury.pl/media/book/txt/dziady-dziadow-czesci-iii-ustep-do-przyjaciol-moskali.txt",
 "https://wolnelektury.pl/media/book/txt/ballady-i-romanse-pani-twardowska.txt",
 "https://wolnelektury.pl/media/book/txt/ballady-i-romanse-powrot-taty.txt",
 "https://wolnelektury.pl/media/book/txt/ballady-i-romanse-switez.txt",
 "https://wolnelektury.pl/media/book/txt/dziady-dziady-poema-dziady-czesc-iv.txt",
 ],
 "Sienkiewicz": [
 "https://wolnelektury.pl/media/book/txt/quo-vadis.txt",
 "https://wolnelektury.pl/media/book/txt/sienkiewicz-we-mgle.txt",
 "https://wolnelektury.pl/media/book/txt/potop-tom-pierwszy.txt",
 "https://wolnelektury.pl/media/book/txt/potop-tom-drugi.txt",
 "https://wolnelektury.pl/media/book/txt/potop-tom-trzeci.txt",
 ],
 "Orzeszkowa": [
 "https://wolnelektury.pl/media/book/txt/orzeszkowa-kto-winien.txt",
 "https://wolnelektury.pl/media/book/txt/nad-niemnem-tom-pierwszy.txt",
 "https://wolnelektury.pl/media/book/txt/nad-niemnem-tom-drugi.txt",
 "https://wolnelektury.pl/media/book/txt/nad-niemnem-tom-trzeci.txt",
 "https://wolnelektury.pl/media/book/txt/gloria-victis-dziwna-historia.txt",
 "https://wolnelektury.pl/media/book/txt/z-pozogi.txt",
 "https://wolnelektury.pl/media/book/txt/pani-dudkowa.txt",
 "https://wolnelektury.pl/media/book/txt/dymy.txt",
 "https://wolnelektury.pl/media/book/txt/syn-stolarza.txt",
 "https://wolnelektury.pl/media/book/txt/dobra-pani.txt",
 "https://wolnelektury.pl/media/book/txt/cnotliwi.txt",
 "https://wolnelektury.pl/media/book/txt/kilka-slow-o-kobietach.txt",
 "https://wolnelektury.pl/media/book/txt/patryotyzm-i-kosmopolityzm.txt",
 "https://wolnelektury.pl/media/book/txt/julianka.txt",
 ],
 "Prus": [
 "https://wolnelektury.pl/media/book/txt/lalka-tom-drugi.txt",
 "https://wolnelektury.pl/media/book/txt/lalka-tom-pierwszy.txt",
 "https://wolnelektury.pl/media/book/txt/antek.txt",
 "https://wolnelektury.pl/media/book/txt/katarynka.txt",
 "https://wolnelektury.pl/media/book/txt/prus-anielka.txt",
 "https://wolnelektury.pl/media/book/txt/prus-placowka.txt",
 
 ],
 "Reymont": [
 "https://wolnelektury.pl/media/book/txt/ziemia-obiecana-tom-pierwszy.txt",
 "https://wolnelektury.pl/media/book/txt/chlopi-czesc-pierwsza-jesien.txt",
 "https://wolnelektury.pl/media/book/txt/reymont-chlopi-zima.txt",
 "https://wolnelektury.pl/media/book/txt/chlopi-czesc-trzecia-wiosna.txt",
 "https://wolnelektury.pl/media/book/txt/chlopi-czesc-czwarta-lato.txt",
 ]
}

In [31]:
def fetch(url):
    file_path = os.path.join("data",os.path.basename(url))
    if os.path.exists(file_path):
        return None, None
    data = request.urlopen(url).read()
    return file_path, data

for author in book_files:
    pool = eventlet.GreenPool()
    
    for file_path, data in pool.imap(fetch, book_files[author]):
        if file_path:
            with open(file_path, mode="wb") as f:
                f.write(data)
print ("DONE")

DONE


In [32]:
def preprocess_file(file_path=None, file_url=None):
    if not file_path and file_url:
        file_path = os.path.join("data",os.path.basename(file_url))
        
    text = open(file_path,'rb').read().decode("utf-8").lower()


    text = regex.sub(u"[^ \n\p{Latin}\-'.?!]", " ",text)
    text = regex.sub(u"[ \n]+", " ", text)
    text = regex.sub(r"----- ta lektura.*","", text)

    return [regex.sub(r"^ ","",l) for l in regex.split('\.|,|\?|!|:',text)]


def get_book_df(document, author):
    return pd.DataFrame({
        'author': pd.Series(len(document)*[author]),
        'txt': pd.Series(document),
    })
    
book_lines_df = pd.concat([
    get_book_df(preprocess_file(file_url=url),author=author) 
        for author in book_files for url in book_files[author] 
])

book_lines_df.head()

,author,txt
0,Mickiewicz,adam mickiewicz dziady
1,Mickiewicz,widowisko część i prawa strona teatru dziewica...
2,Mickiewicz,dziewica wstaje od stołu świeco niedobra
3,Mickiewicz,właśnie pora była zgasnąć
4,Mickiewicz,i nie mogłam doczytać czyż podobna zasnąć


In [33]:
book_lines_df.groupby('author').count()

,txt
author,
Mickiewicz,1206
Orzeszkowa,22177
Prus,31033
Reymont,24107
Sienkiewicz,40381


In [34]:
book_lines_df['words'] = book_lines_df['txt'].apply(
    lambda x: len(x.split())
)
book_lines_df.groupby('author')['words'].describe()

,count,mean,std,min,25%,50%,75%,max
author,,,,,,,,
Mickiewicz,1206.0,11.466003,10.459886,0.0,4.0,8.0,16.0,140.0
Orzeszkowa,22177.0,19.400821,17.257917,0.0,6.0,15.0,27.0,219.0
Prus,31033.0,12.131537,10.041450,0.0,5.0,10.0,17.0,133.0
Reymont,24107.0,16.359398,18.880965,0.0,5.0,10.0,21.0,316.0
Sienkiewicz,40381.0,13.705579,12.038426,0.0,5.0,10.0,19.0,146.0


In [35]:
book_lines_df.groupby('author')['words'].quantile(0.98)

author
Mickiewicz     40.0
Orzeszkowa     67.0
Prus           39.0
Reymont        72.0
Sienkiewicz    47.0
Name: words, dtype: float64

In [36]:
book_lines_df=book_lines_df[~(book_lines_df['words']==0)]
book_lines_df.groupby('author')['words'].describe()

,count,mean,std,min,25%,50%,75%,max
author,,,,,,,,
Mickiewicz,1198.0,11.542571,10.452555,1.0,4.0,8.0,16.0,140.0
Orzeszkowa,22087.0,19.479875,17.248458,1.0,7.0,15.0,27.0,219.0
Prus,30941.0,12.167609,10.034522,1.0,5.0,10.0,17.0,133.0
Reymont,24079.0,16.378421,18.883692,1.0,5.0,10.0,21.0,316.0
Sienkiewicz,40258.0,13.747454,12.032905,1.0,5.0,10.0,19.0,146.0


In [37]:
train_df, test_df = train_test_split(book_lines_df,test_size=0.1,stratify=book_lines_df['author'],)

In [38]:
vectorizer = CountVectorizer()
vectorizer.fit(train_df['txt'])
sample_sentence = train_df.iloc[10]['txt']
print ("Extracting vector for sentence: '{}'".format(sample_sentence))
vectorizer.transform([sample_sentence])

Extracting vector for sentence: 'żydoskie paroby'


<1x127697 sparse matrix of type '<class 'numpy.int64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [39]:
X_train=vectorizer.transform(train_df['txt'])
X_test=vectorizer.transform(test_df['txt'])
model = LogisticRegression(class_weight='balanced', dual=True)
model.fit(X_train, train_df['author'])

/Users/HeX/Library/Python/3.7/lib/python/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/HeX/Library/Python/3.7/lib/python/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/HeX/Library/Python/3.7/lib/python/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=True,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [40]:
model.score(X_test, test_df['author'])

0.7764189929999157

In [41]:
target = test_df['author']
predicted = model.predict(X_test)
print (metrics.classification_report(target, predicted, digits=4))


              precision    recall  f1-score   support

  Mickiewicz     0.3562    0.4333    0.3910       120
  Orzeszkowa     0.7582    0.7252    0.7413      2209
        Prus     0.7602    0.7450    0.7525      3094
     Reymont     0.7730    0.7566    0.7647      2408
 Sienkiewicz     0.8137    0.8507    0.8318      4026

    accuracy                         0.7764     11857
   macro avg     0.6923    0.7022    0.6963     11857
weighted avg     0.7765    0.7764    0.7762     11857

